# Hybrid Search using RRF

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/elastic/elasticsearch-labs/blob/main/notebooks/search/02-hybrid-search.ipynb)

In this example we'll use the reciprocal rank fusion algorithm to combine the results of BM25 and kNN semantic search.
We'll use the same dataset we used in our [quickstart](https://github.com/elastic/elasticsearch-labs/blob/main/notebooks/search/00-quick-start.ipynb) guide.
You can use RRF for hybrid search out of the box, without any additional configuration.

We also provide a walkthrough of a toy example, which demonstrates how RRF ranking works at a basic level.

# 🧰 Requirements

For this example, you will need:

- An Elastic deployment with minimum **4GB machine learning node**
   - We'll be using [Elastic Cloud](https://www.elastic.co/guide/en/cloud/current/ec-getting-started.html) for this example (available with a [free trial](https://cloud.elastic.co/registration?elektra=en-ess-sign-up-page))


# Create Elastic Cloud deployment

If you don't have an Elastic Cloud deployment, sign up [here](https://cloud.elastic.co/registration?fromURI=%2Fhome) for a free trial.

- Go to the [Create deployment](https://cloud.elastic.co/deployments/create) page
   - Under **Advanced settings**, go to **Machine Learning instances**
   - You'll need at least **4GB** RAM per zone for this tutorial
   - Select **Create deployment**

# Install packages and initialize the Elasticsearch Python client

To get started, we'll need to connect to our Elastic deployment using the Python client.
Because we're using an Elastic Cloud deployment, we'll use the **Cloud ID** to identify our deployment.

First we need to `pip` install the packages we need for this example.

In [7]:
!pip install elasticsearch>=8.9.0
!pip install sentence_transformers
!pip install torch


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
zsh:1: 8.9.0 not found
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)

[notice] A new release of pip is available: 23.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` bef

Next we need to import the `elasticsearch` module and the `getpass` module.
`getpass` is part of the Python standard library and is used to securely prompt for credentials.

In [2]:
from elasticsearch import Elasticsearch, helpers
from urllib.request import urlopen
import getpass
from sentence_transformers import SentenceTransformer
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = SentenceTransformer('all-MiniLM-L6-v2', device=device)

/Users/joe/.pyenv/versions/3.10.3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Now we can instantiate the Python Elasticsearch client.
First we prompt the user for their password and Cloud ID.

🔐 NOTE: `getpass` enables us to securely prompt the user for credentials without echoing them to the terminal, or storing it in memory.

Then we create a `client` object that instantiates an instance of the `Elasticsearch` class.

In [3]:
# Found in the 'Manage Deployment' page
CLOUD_ID = getpass.getpass('Enter Elastic Cloud ID:  ')

# Password for the 'elastic' user generated by Elasticsearch
ELASTIC_PASSWORD = getpass.getpass('Enter Elastic password:  ')

# Create the client instance
client = Elasticsearch(
    cloud_id=CLOUD_ID,
    basic_auth=("elastic", ELASTIC_PASSWORD)
)

Confirm that the client has connected with this test

In [4]:
print(client.info())

{'name': 'instance-0000000007', 'cluster_name': 'd1bd36862ce54c7b903e2aacd4cd7f0a', 'cluster_uuid': 'tIkh0X_UQKmMFQKSfUw-VQ', 'version': {'number': '8.9.0', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '8aa461beb06aa0417a231c345a1b8c38fb498a0d', 'build_date': '2023-07-19T14:43:58.555259655Z', 'build_snapshot': False, 'lucene_version': '9.7.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


Refer to https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html#connect-self-managed-new to learn how to connect to a self-managed deployment.

Read https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html#connect-self-managed-new to learn how to connect using API keys.


# Create Elasticsearch index with required mappings

We need to add a field to support dense vector storage and search.
Note the `title_vector` field below, which is used to store the dense vector representation of the `title` field.

In [5]:
# Create the index
client.indices.create(
    index='rrf_book_index', 
    mappings={
        "properties": {
            "title": {"type": "text"},
            "authors": {"type": "keyword"},
            "summary": {"type": "text"},
            "publish_date": {"type": "date"},
            "num_reviews": {"type": "integer"},
            "publisher": {"type": "keyword"},
            "title_vector": { 
                "type": "dense_vector", 
                "dims": 384, 
                "index": "true", 
                "similarity": "dot_product" 
            }
        }
    })


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'rrf_book_index'})

## Dataset

Let's index some data.
Note that we are embedding the `title` field using the sentence transformer model.
Once indexed, you'll see that your documents contain a `title_vector` field (`"type": "dense_vector"`) which contains a vector of floating point values.
This is the embedding of the `title` field in vector space.
We'll use this field to perform semantic search using kNN.

In [6]:
books = [
    {
        "title": "The Pragmatic Programmer: Your Journey to Mastery",
        "authors": ["andrew hunt", "david thomas"],
        "summary": "A guide to pragmatic programming for software engineers and developers",
        "publish_date": "2019-10-29",
        "num_reviews": 30,
        "publisher": "addison-wesley"
    },
    {
        "title": "Python Crash Course",
        "authors": ["eric matthes"],
        "summary": "A fast-paced, no-nonsense guide to programming in Python",
        "publish_date": "2019-05-03",
        "num_reviews": 42,
        "publisher": "no starch press"
    },
    {
        "title": "Artificial Intelligence: A Modern Approach",
        "authors": ["stuart russell", "peter norvig"],
        "summary": "Comprehensive introduction to the theory and practice of artificial intelligence",
        "publish_date": "2020-04-06",
        "num_reviews": 39,
        "publisher": "pearson"
    },
    {
        "title": "Clean Code: A Handbook of Agile Software Craftsmanship",
        "authors": ["robert c. martin"],
        "summary": "A guide to writing code that is easy to read, understand and maintain",
        "publish_date": "2008-08-11",
        "num_reviews": 55,
        "publisher": "prentice hall"
    },
    {
        "title": "You Don't Know JS: Up & Going",
        "authors": ["kyle simpson"],
        "summary": "Introduction to JavaScript and programming as a whole",
        "publish_date": "2015-03-27",
        "num_reviews": 36,
        "publisher": "oreilly"
    },
    {
        "title": "Eloquent JavaScript",
        "authors": ["marijn haverbeke"],
        "summary": "A modern introduction to programming",
        "publish_date": "2018-12-04",
        "num_reviews": 38,
        "publisher": "no starch press"
    },
    {
        "title": "Design Patterns: Elements of Reusable Object-Oriented Software",
        "authors": ["erich gamma", "richard helm", "ralph johnson", "john vlissides"],
        "summary": "Guide to design patterns that can be used in any object-oriented language",
        "publish_date": "1994-10-31",
        "num_reviews": 45,
        "publisher": "addison-wesley"
    },
    {
        "title": "The Clean Coder: A Code of Conduct for Professional Programmers",
        "authors": ["robert c. martin"],
        "summary": "A guide to professional conduct in the field of software engineering",
        "publish_date": "2011-05-13",
        "num_reviews": 20,
        "publisher": "prentice hall"
    },
    {
        "title": "JavaScript: The Good Parts",
        "authors": ["douglas crockford"],
        "summary": "A deep dive into the parts of JavaScript that are essential to writing maintainable code",
        "publish_date": "2008-05-15",
        "num_reviews": 51,
        "publisher": "oreilly"
    },
    {
        "title": "Introduction to the Theory of Computation",
        "authors": ["michael sipser"],
        "summary": "Introduction to the theory of computation and complexity theory",
        "publish_date": "2012-06-27",
        "num_reviews": 33,
        "publisher": "cengage learning"
    },
]

## Index documents

Our dataset is a Python list that contains dictionaries of movie titles and descriptions.
We'll use the `helpers.bulk` method to index our documents in batches.

The following code iterates over the list of books and creates a list of actions to be performed.
Each action is a dictionary containing an "index" operation on our Elasticsearch index.
The book's title is encoded using our selected model, and the encoded vector is added to the book document.
The book document is then added to the list of actions.

Finally, we call the `bulk` method, specifying the index name and the list of actions.

In [7]:
actions = []
for book in books:
    actions.append({"index": {"_index": "rrf_book_index"}})
    titleEmbedding = model.encode(book["title"]).tolist()
    book["title_vector"] = titleEmbedding
    actions.append(book)

client.bulk(index="rrf_book_index", operations=actions)

ObjectApiResponse({'took': 61, 'errors': False, 'items': [{'index': {'_index': 'rrf_book_index', '_id': 'W72R-YkBh2PvFlUSVfHi', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'rrf_book_index', '_id': 'XL2R-YkBh2PvFlUSVfHi', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'rrf_book_index', '_id': 'Xb2R-YkBh2PvFlUSVfHi', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'rrf_book_index', '_id': 'Xr2R-YkBh2PvFlUSVfHi', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 3, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'rrf_book_index', '_id': 'X72R-YkBh2PvFlUSVfHi', '_version': 1, 'result':

## Pretty printing Elasticsearch responses

This is a helper function to print Elasticsearch responses in a readable format.

In [10]:
def pretty_response(response):
    for hit in response['hits']['hits']:
        id = hit['_id']
        publication_date = hit['_source']['publish_date']
        rank = hit['_rank']
        title = hit['_source']['title']
        summary = hit['_source']['summary']
        pretty_output = (f"\nID: {id}\nPublication date: {publication_date}\nTitle: {title}\nSummary: {summary}\nRank: {rank}")
        print(pretty_output)

# Querying Documents with Hybrid Search

Now we need to perform a query using two different search strategies:
- Semantic search using the "all-MiniLM-L6-v2" embedding model
- Keyword search using the "title" field

We then use [Reciprocal Rank Fusion (RRF)](https://www.elastic.co/guide/en/elasticsearch/reference/current/rrf.html) to balance the scores to provide a final list of documents, ranked in order of relevance. RRF is a ranking algorithm for combining results from different information retrieval strategies.

**NOTE** Note that _score is null, and we instead use _rank to show our top-ranked documents.

In [12]:
response = client.search(
    index="rrf_book_index", 
    size=5, 
    query={
        "match": {
            "summary": "python programming"
        }
    }, 
    knn={
        "field": "title_vector",
        "query_vector" : model.encode("python programming").tolist(), # generate embedding for query so it can be compared to `title_vector`
        "k": 5,
        "num_candidates": 10
    },
    rank={
        "rrf": {}
    }
)

pretty_response(response)


ID: XL2R-YkBh2PvFlUSVfHi
Publication date: 2019-05-03
Title: Python Crash Course
Summary: A fast-paced, no-nonsense guide to programming in Python
Rank: 1

ID: W72R-YkBh2PvFlUSVfHi
Publication date: 2019-10-29
Title: The Pragmatic Programmer: Your Journey to Mastery
Summary: A guide to pragmatic programming for software engineers and developers
Rank: 2

ID: YL2R-YkBh2PvFlUSVfHi
Publication date: 2018-12-04
Title: Eloquent JavaScript
Summary: A modern introduction to programming
Rank: 3

ID: X72R-YkBh2PvFlUSVfHi
Publication date: 2015-03-27
Title: You Don't Know JS: Up & Going
Summary: Introduction to JavaScript and programming as a whole
Rank: 4

ID: ZL2R-YkBh2PvFlUSVfHi
Publication date: 2012-06-27
Title: Introduction to the Theory of Computation
Summary: Introduction to the theory of computation and complexity theory
Rank: 5
